# Team 1 - Final Project
## Jose Sandoval, Dheemanth Rajakumar, and Israel Romero Olvera
### Test 1

In [1]:
#Importing the required libraries
import os
import numpy as np
import pretty_midi
import tensorflow as tf

In [2]:
#Preparing the directories to load the Midi files
rootdir=r'C:\Users\isral\AAI_511\Composer_Dataset\Composer_Dataset\NN_midi_files_extended'
traindir = os.path.join(rootdir,'train') #Training set
testdir = os.path.join(rootdir,'test') #Testing set
valdir = os.path.join(rootdir,'dev') #Validation set

In [32]:
#Creating a class to handle the MIDI loading
class MidiDataset:
    def __init__(self, train_dir, test_dir, seq_len=128, fs=10):
        """
        train_dir: root folder with subfolders for each composer
        seq_len: number of time steps per piano roll
        fs: frames per second for piano roll
        """
        self.train_dir = train_dir
        self.test_dir = test_dir
        self.val_dir = ""
        self.seq_len = seq_len
        self.fs = fs
        self.train_samples = []
        self.train_labels = []
        self.train_label_to_idx = {}
        self.test_samples = []
        self.test_labels = []
        self.test_label_to_idx = {}
        self.val_samples = []
        self.val_labels = []
        self.val_label_to_idx = {}
        self._load_data()
    def _load_data(self):
        self.train_labels, self.train_label_to_idx, self.train_samples = self.load_path(self.train_dir)
        self.test_labels, self.test_label_to_idx, self.test_samples = self.load_path(self.test_dir)
    def load_path(self, path):
        labels = []
        label_to_idx = {}
        samples = []
        #First, we load all the subfolders list, which represents the authors list
        composer_dirs = [
            directory for directory in os.listdir(self.train_dir)
            if os.path.isdir(os.path.join(self.train_dir, directory))
        ]
        #Next, we load each individual midi file
        for idx, composer in enumerate(composer_dirs):
            label_to_idx[composer] = idx #Here we create a unique ID for each composer
            #Now, let's load all individual files in each composer's folder
            composer_path = os.path.join(path, composer)
            for fname in os.listdir(composer_path):
                if fname.endswith('.mid') or fname.endswith('.midi'): #Including any .MID or .MIDI files
                    file_path = os.path.join(composer_path, fname)
                    samples.append(file_path)
                    labels.append(idx)
        return labels, label_to_idx, samples
    #This function will convert the midi notes into a piano roll. Only pitch and duration of notes will be preserved, but other attributes like note velocity or midi channel will be lost.
    def midi_to_pianoroll(self, midi_path):
        #Loading the midi file into memory
        midi_data = pretty_midi.PrettyMIDI(midi_path)
        #Converting the midi data into a piano roll. By default, the velocity values are preserved
        piano_roll = midi_data.get_piano_roll(fs=self.fs)
        #Make sure piano roll is binary: note active (1) or inactive (0), losing the velocity attribute: we just care if the note is on or off.
        piano_roll = (piano_roll > 0).astype(np.float32)
        # Transpose to (time, pitch), as TensorFlow will use the first dimension to establish sequenciality
        piano_roll = piano_roll.T

        # Pad or truncate to seq_len, to ensure all tensors are the same length (part of the melody might be lost, but the sample of the song should be good enough for classification.
        if piano_roll.shape[0] < self.seq_len:
            pad_width = self.seq_len - piano_roll.shape[0]
            piano_roll = np.pad(piano_roll, ((0, pad_width), (0, 0)))
        else:
            piano_roll = piano_roll[:self.seq_len]

        # Keep only 0-127 notes, losing other events like tempo changes, instrument changes, pitch bend, etc.
        # NOTE: while foot switch messages get excluded too, the note duration is preserved in the piano roll as the note state is captured as "ON" by PrettyMIDI.
        piano_roll = piano_roll[:, :128]

        return piano_roll

    def generator(self, samples, labels):
        for midi_path, label in zip(self.train_samples, self.train_labels):
            piano_roll = self.midi_to_pianoroll(midi_path)
            yield piano_roll, label
    #This function builds the TensorFlow dataset
    def get_tf_dataset(self, samples, labels, batch_size=16, shuffle=True):
        output_signature = (
            tf.TensorSpec(shape=(self.seq_len, 128), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
        dataset = tf.data.Dataset.from_generator(
            lambda: self.generator(samples, labels),
            output_signature=output_signature
        )
        #The shuffle parameter will help the Neural Network training by shuffling the sample's order, preventing the model from seeing samples in the same order every epoch
        if shuffle:
            dataset = dataset.shuffle(buffer_size=len(self.train_samples))

        dataset = dataset.batch(batch_size)
        return dataset
    def get_tf_dataset_train(self, batch_size=16, shuffle=True):
        return self.get_tf_dataset(self.train_samples, self.train_labels, batch_size, shuffle)
    def get_tf_dataset_test(self, batch_size=16, shuffle=True):
        return self.get_tf_dataset(self.test_samples, self.test_labels, batch_size, shuffle)
    def load_validation_data(self, val_dir, batch_size=16, shuffle=True):
        self.val_dir = val_dir
        self.val_labels, self.val_label_to_idx, self.val_samples = self.load_path(val_dir)
        return self.get_tf_dataset(self.val_samples, self.val_labels, batch_size, shuffle)

In [35]:
dataset_builder = MidiDataset(train_dir=traindir, test_dir=testdir, seq_len=128, fs=10)
train_dataset = dataset_builder.get_tf_dataset_train(batch_size=16)
test_dataset = dataset_builder.get_tf_dataset_test(batch_size=16)

val_dataset = dataset_builder.load_validation_data(val_dir=valdir, batch_size=16)

In [42]:
for piano_rolls, labels in train_dataset.take(2):
    print('Batch piano_rolls shape:', piano_rolls.shape)  # (batch_size, seq_len, 128)
    print('Batch labels:', labels)                        # (batch_size,)
    print('Labels batch shape:', labels.shape)

for piano_rolls, labels in test_dataset.take(2):
    print('Batch piano_rolls shape:', piano_rolls.shape)  # (batch_size, seq_len, 128)
    print('Batch labels:', labels)                        # (batch_size,)
    print('Labels batch shape:', labels.shape)
for piano_rolls, labels in val_dataset.take(2):
    print('Batch piano_rolls shape:', piano_rolls.shape)  # (batch_size, seq_len, 128)
    print('Batch labels:', labels)                        # (batch_size,)
    print('Labels batch shape:', labels.shape)


Batch piano_rolls shape: (16, 128, 128)
Batch labels: tf.Tensor([3 2 3 6 6 1 6 7 3 1 4 4 0 6 8 8], shape=(16,), dtype=int32)
Labels batch shape: (16,)
Batch piano_rolls shape: (16, 128, 128)
Batch labels: tf.Tensor([5 5 8 4 4 2 5 0 2 7 3 8 6 6 0 2], shape=(16,), dtype=int32)
Labels batch shape: (16,)
Batch piano_rolls shape: (16, 128, 128)
Batch labels: tf.Tensor([3 7 8 8 3 0 7 3 4 0 8 4 3 5 6 4], shape=(16,), dtype=int32)
Labels batch shape: (16,)
Batch piano_rolls shape: (16, 128, 128)
Batch labels: tf.Tensor([4 2 3 4 6 4 4 0 2 1 6 2 7 3 7 2], shape=(16,), dtype=int32)
Labels batch shape: (16,)
Batch piano_rolls shape: (16, 128, 128)
Batch labels: tf.Tensor([6 3 1 5 1 1 5 4 1 0 5 0 5 8 7 1], shape=(16,), dtype=int32)
Labels batch shape: (16,)
Batch piano_rolls shape: (16, 128, 128)
Batch labels: tf.Tensor([2 7 3 6 3 6 6 1 8 3 3 5 7 2 5 4], shape=(16,), dtype=int32)
Labels batch shape: (16,)


In [41]:
print(train_dataset.element_spec)
print(test_dataset.element_spec)
print(val_dataset.element_spec)

(TensorSpec(shape=(None, 128, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 128, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 128, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [43]:
sample_count = 0
for piano_rolls, labels in val_dataset:
    sample_count += piano_rolls.shape[0]  # Number of samples in this batch

print("Total samples:", sample_count)

Total samples: 369
